In [1]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sys

sys.path.append("/code")

from tqdm import tqdm
import torch
# device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# import gym
# import recogym

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
import optuna
# from memory_profiler import profile


from estimators import (
    DirectMethod as DM
)

from simulation_utils import (
    eval_policy,
    generate_dataset,
    create_simulation_data_from_pi,
    get_train_data,
    get_opl_results_dict,
    CustomCFDataset,
    calc_reward
)

from models import (    
    CFModel,
    NeighborhoodModel,
    BPRModel, 
    RegressionModel
)

from training_utils import (
    fit_bpr,
    train,
    validation_loop
 )

from custom_losses import (
    SNDRPolicyLoss,
    BPRLoss
    )

random_state=12345
random_ = check_random_state(random_state)

Using device: cpu
Using device: cpu
Using device: cpu


In [2]:
pd.options.display.float_format = '{:,.4f}'.format

## `trainer_trial` Function

This function runs policy learning experiments using offline bandit data and evaluates various estimators.

### Parameters
- **num_runs** (int): Number of experimental runs per training size
- **num_neighbors** (int): Number of neighbors to consider in the neighborhood model
- **num_rounds_list** (list): List of training set sizes to evaluate
- **dataset** (dict): Contains dataset information including embeddings, action probabilities, and reward probabilities
- **batch_size** (int): Batch size for training the policy model
- **num_epochs** (int): Number of training epochs for each experiment
- **lr** (float, default=0.001): Learning rate for the optimizer

### Process Flow
1. Initializes result structures and retrieval models
2. For each training size in `num_rounds_list`:
   - Creates a uniform logging policy and simulates data
   - Generates training data for offline learning
   - Fits regression and neighborhood models for reward estimation
   - Initializes and trains a counterfactual policy model
   - Evaluates policy performance using various estimators
   - Collects metrics on policy reward and embedding quality

### Returns
- **DataFrame**: Results table with rows indexed by training size and columns for various metrics:
  - `policy_rewards`: True expected reward of the learned policy
  - Various estimator errors (`ipw`, `reg_dm`, `conv_dm`, `conv_dr`, `conv_sndr`)
  - Variance metrics for each estimator
  - Embedding quality metrics comparing learned representations to ground truth

### Implementation Notes
- Uses uniform random logging policy for collecting offline data
- Employs Self-Normalized Doubly Robust (SNDR) policy learning
- Measures embedding quality via RMSE to original/ground truth embeddings

In [ ]:
def trainer_trial(
                  num_runs,
                  num_neighbors,
                  num_rounds_list,
                  dataset,
                  batch_size,
                  val_size=2000
                  ):
    
    # Define device at the beginning
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def objective(trial):
    
        # Optuna objective function
        lr = trial.suggest_loguniform("lr", 1e-4, 1e-3)
        epochs = trial.suggest_int("num_epochs", 1, 10)

        trial_neigh_model = NeighborhoodModel(
                                                train_data['x_idx'],
                                                train_data['a'], 
                                                our_a,
                                                our_x, 
                                                train_data['r'], 
                                                num_neighbors=num_neighbors
                                            )
        

        trial_model = CFModel(
                        n_users, 
                        n_actions, 
                        emb_dim, 
                        initial_user_embeddings=torch.tensor(our_x, device=device), 
                        initial_actions_embeddings=torch.tensor(our_a, device=device)
                        )
        
        # Training
        train(trial_model, train_loader, trial_neigh_model, criterion=SNDRPolicyLoss(), num_epochs=epochs, lr=lr, device=device)
        return validation_loop(trial_model, val_loader, trial_neigh_model)
    
    device = torch.device('cpu')

    dm = DM()
    results = {}

    our_x, our_a = dataset["our_x"], dataset["our_a"]
    emb_x, emb_a = dataset["emb_x"], dataset["emb_a"]
    original_x, original_a = dataset["original_x"], dataset["original_a"]
    n_users, n_actions, emb_dim = dataset["n_users"], dataset["n_actions"], dataset["emb_dim"]
    
    first = True

    for train_size in num_rounds_list:
        reg_results, conv_results = [], []
        
        for run in range(num_runs):

            pi_0 = softmax(our_x @ our_a.T, axis=1)
            original_policy_prob = np.expand_dims(pi_0, -1)

            simulation_data = create_simulation_data_from_pi(
                                                            dataset,
                                                            pi_0,
                                                            train_size + val_size,
                                                            random_state=(run + 1) * train_size
                                                            )
            # test_data = get_test_data(dataset, simulation_data, n_test_data)

            # idx = np.arange(train_size) + n_test_data
            idx = np.arange(train_size)

            train_data = get_train_data(n_actions, train_size, simulation_data, idx, our_x)
            val_data = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size) + train_size, our_x)

            regression_model = RegressionModel(
                                    n_actions=n_actions,
                                    action_context=our_x,
                                    base_model=LogisticRegression(random_state=12345)
                                    )
            
            regression_model.fit(
                                train_data['x'], 
                                train_data['a'],
                                train_data['r'],
                                original_policy_prob[train_data['x_idx'],
                                train_data['a']].squeeze()
                                )
            
            neighberhoodmodel = NeighborhoodModel(
                                                    train_data['x_idx'],
                                                    train_data['a'], 
                                                    our_a,
                                                    our_x, 
                                                    train_data['r'], 
                                                    num_neighbors=num_neighbors
                                                )
            

            model = CFModel(
                            n_users, 
                            n_actions, 
                            emb_dim, 
                            initial_user_embeddings=torch.tensor(our_x, device=device), 
                            initial_actions_embeddings=torch.tensor(our_a, device=device)
                            )
            
            cf_dataset =  CustomCFDataset(
                                       train_data['x_idx'], 
                                       train_data['a'], 
                                       train_data['r'], 
                                       original_policy_prob
                                       )
            
            train_loader = DataLoader(cf_dataset, batch_size=batch_size, shuffle=True)

            val_dataset =  CustomCFDataset(
                            val_data['x_idx'], 
                            val_data['a'], 
                            val_data['r'], 
                            original_policy_prob
                            )
            
            val_loader = DataLoader(val_dataset, batch_size=val_size, shuffle=True)

            if first:
                policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
                conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob, policy))
                conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
                # bpr_scores = bpr_model.calc_scores(torch.tensor(train_data['x_idx'], device=device, dtype=torch.long)).detach().cpu().numpy()
                reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], regression_model.predict(train_data['x']))
                reg_results.append(reg_dm)
                first = False
                reg_results = np.array(reg_results)
                conv_results = np.array(conv_results)
                results[0] = get_opl_results_dict(reg_results, conv_results)
                reg_results, conv_results = [], []
                
            # Bloss = BPRLoss()
            # bpr_model = BPRModel(
            #                     n_users,
            #                     n_actions,
            #                     emb_x.shape[1], 
            #                     initial_user_embeddings=torch.tensor(our_x, device=device), 
            #                     initial_actions_embeddings=torch.tensor(our_a, device=device)
            #                     )

            study = optuna.create_study(direction="maximize")
            study.optimize(objective, n_trials=10, show_progress_bar=True)
            best_params = study.best_params

            neighberhoodmodel = NeighborhoodModel(
                                                    train_data['x_idx'],
                                                    train_data['a'], 
                                                    our_a,
                                                    our_x, 
                                                    train_data['r'], 
                                                    num_neighbors=num_neighbors
                                                )
            

            model = CFModel(
                            n_users, 
                            n_actions, 
                            emb_dim, 
                            initial_user_embeddings=torch.tensor(our_x, device=device), 
                            initial_actions_embeddings=torch.tensor(our_a, device=device)
                            )

            train(model, train_loader, neighberhoodmodel, criterion=SNDRPolicyLoss(), num_epochs=best_params['num_epochs'], lr=best_params['lr'], device=device)
            # fit_bpr(bpr_model, Bloss, train_loader, num_epochs=3, lr=0.001, device=device)
            # neighborhood_model.update(model.get_params()[0].detach().numpy(), model.get_params()[1].detach().numpy())'

            our_x, our_a = model.get_params()
            our_a, our_x = our_a.detach().cpu().numpy(), our_x.detach().cpu().numpy()

            policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)

            # bpr_scores = bpr_model.calc_scores(torch.tensor(train_data['x_idx'], device=device, dtype=torch.long)).detach().cpu().numpy()
            # reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], bpr_scores)
            reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], regression_model.predict(train_data['x']))

            reg_results.append(reg_dm)

            # conv_results.append(eval_policy(neighberhoodmodel, test_data, original_policy_prob[test_data['x_idx']], policy))
            conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob, policy))

            conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])

            # temp.append(np.mean((emb_a-our_a)**2, axis=0))

            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
            
            our_a, our_x = original_a.copy(), original_x.copy()

        reg_results = np.array(reg_results)
        conv_results = np.array(conv_results)

        results[train_size] = get_opl_results_dict(reg_results, conv_results)
    
    return pd.DataFrame.from_dict(results, orient='index')

## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [5]:
num_runs = 1

In [6]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 4,
                    # sigma = 0.1,
                    eps = 0.2, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.02
                    )

train_dataset = generate_dataset(dataset_params)

CTR: 0.01861246403332251


In [7]:
train_dataset.keys()

dict_keys(['emb_a', 'our_a', 'original_a', 'emb_x', 'our_x', 'original_x', 'q_x_a', 'n_actions', 'n_users', 'emb_dim', 'user_prior'])

In [8]:
num_runs = 1
batch_size = 200
num_neighbors = 51
num_rounds_list = [30000, 60000, 80000, 90000]

### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [ ]:
df4 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=35000)

[I 2025-08-18 00:46:01,982] A new study created in memory with name: no-name-1686133b-49dd-4658-8ffa-e01ba0620026
Best trial: 0. Best value: 0.0103668:  10%|█         | 1/10 [03:28<31:19, 208.79s/it]

[I 2025-08-18 00:49:30,770] Trial 0 finished with value: 0.01036684519920781 and parameters: {'lr': 0.00021998095044840113, 'num_epochs': 1}. Best is trial 0 with value: 0.01036684519920781.


Best trial: 0. Best value: 0.0103668:  20%|██        | 2/10 [03:34<11:55, 89.45s/it] 

[I 2025-08-18 00:49:36,682] Trial 2 finished with value: 0.010365837663681828 and parameters: {'lr': 0.00020944807542242172, 'num_epochs': 3}. Best is trial 0 with value: 0.01036684519920781.
[I 2025-08-18 00:49:36,687] Trial 1 finished with value: 0.010363003040643437 and parameters: {'lr': 0.0006597293469526339, 'num_epochs': 3}. Best is trial 0 with value: 0.01036684519920781.


Best trial: 0. Best value: 0.0103668:  40%|████      | 4/10 [06:47<09:22, 93.71s/it]

...

Best trial: 3. Best value: 0.0105873:  90%|█████████ | 9/10 [23:20<01:52, 112.65s/it]

[I 2025-08-18 01:25:16,393] Trial 8 finished with value: 0.010488506759005475 and parameters: {'lr': 0.0002499885177096407, 'num_epochs': 5}. Best is trial 3 with value: 0.010587282562991815.


Best trial: 3. Best value: 0.0105873: 100%|██████████| 10/10 [25:38<00:00, 153.86s/it]


[I 2025-08-18 01:27:34,040] Trial 9 finished with value: 0.007912739149983753 and parameters: {'lr': 0.0008247421354429188, 'num_epochs': 4}. Best is trial 3 with value: 0.010587282562991815.


[I 2025-08-18 01:34:06,011] A new study created in memory with name: no-name-94cc3a01-6135-401e-a646-36bfea6a84d3
  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.0198,0.0202,0.0200,0.0198,0.0200,0.0205,0.2784,0.0000,0.2374,0.0000
30000,0.0199,0.0205,0.0198,0.0173,0.0195,0.0199,0.4736,0.3790,0.3557,0.3496
60000,0.0199,0.0170,0.0197,0.0221,0.0176,0.0166,0.4546,0.3624,0.3900,0.3823
80000,0.0198,0.0198,0.0194,0.0189,0.0195,0.0202,0.2979,0.1149,0.2460,0.1062
90000,0.0199,0.0205,0.0191,0.0192,0.0199,0.0200,0.6269,0.5273,0.6198,0.6548


In [ ]:
df4

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.0198,0.0202,0.0200,0.0198,0.0200,0.0205,0.0000,0.0000,0.0000,0.0000,0.0000,0.2784,0.0000,0.2374,0.0000
30000,0.0199,0.0205,0.0198,0.0173,0.0195,0.0199,0.0000,0.0000,0.0000,0.0000,0.0000,0.4736,0.3790,0.3557,0.3496
60000,0.0199,0.0170,0.0197,0.0221,0.0176,0.0166,0.0000,0.0000,0.0000,0.0000,0.0000,0.4546,0.3624,0.3900,0.3823
80000,0.0198,0.0198,0.0194,0.0189,0.0195,0.0202,0.0000,0.0000,0.0000,0.0000,0.0000,0.2979,0.1149,0.2460,0.1062
90000,0.0199,0.0205,0.0191,0.0192,0.0199,0.0200,0.0000,0.0000,0.0000,0.0000,0.0000,0.6269,0.5273,0.6198,0.6548


In [ ]:
num_rounds_list = [3000, 6000, 8000, 9000]

### 2

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.001$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [ ]:
df5 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=3500)

[I 2025-08-17 18:59:47,813] A new study created in memory with name: no-name-da278b4b-2e05-4978-b549-0ee7a31d906a
Best trial: 0. Best value: 0.0055823:  10%|█         | 1/10 [00:13<01:58, 13.20s/it]

[I 2025-08-17 19:00:01,010] Trial 0 finished with value: 0.005582304033634449 and parameters: {'lr': 0.00013239533643429304, 'num_epochs': 3}. Best is trial 0 with value: 0.005582304033634449.
[I 2025-08-17 19:00:01,210] Trial 1 finished with value: 0.005557379902794693 and parameters: {'lr': 0.0005547548817794695, 'num_epochs': 5}. Best is trial 0 with value: 0.005582304033634449.


Best trial: 0. Best value: 0.0055823:  30%|███       | 3/10 [00:25<01:01,  8.75s/it]

[I 2025-08-17 19:00:13,770] Trial 2 finished with value: 0.00556623068617111 and parameters: {'lr': 0.000718531569169898, 'num_epochs': 2}. Best is trial 0 with value: 0.005582304033634449.


Best trial: 0. Best value: 0.0055823:  40%|████      | 4/10 [00:26<00:34,  5.68s/it]

...

[I 2025-08-17 19:11:30,806] Trial 7 finished with value: 0.009127868076393845 and parameters: {'lr': 0.0006385589919752676, 'num_epochs': 6}. Best is trial 5 with value: 0.009140427565767958.


Best trial: 5. Best value: 0.00914043:  90%|█████████ | 9/10 [03:30<00:23, 23.65s/it]

[I 2025-08-17 19:11:59,451] Trial 8 finished with value: 0.009113317619059215 and parameters: {'lr': 0.00016551277217555124, 'num_epochs': 2}. Best is trial 5 with value: 0.009140427565767958.


Best trial: 5. Best value: 0.00914043: 100%|██████████| 10/10 [03:43<00:00, 22.34s/it]


[I 2025-08-17 19:12:12,589] Trial 9 finished with value: 0.009119895306215898 and parameters: {'lr': 0.0008896592064285964, 'num_epochs': 8}. Best is trial 5 with value: 0.009140427565767958.


In [ ]:
df5

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.0198,0.0191,0.0196,0.0197,0.0197,0.0196,0.0000,0.0000,0.0000,0.0000,0.0000,0.2784,0.0000,0.2374,0.0000
3000,0.0198,0.0191,0.0196,0.0197,0.0197,0.0196,0.0000,0.0000,0.0000,0.0000,0.0000,0.2783,0.0006,0.2374,0.0004
6000,0.0198,0.0225,0.0224,0.0229,0.0226,0.0215,0.0000,0.0000,0.0000,0.0000,0.0000,0.2780,0.0037,0.2375,0.0024
8000,0.0198,0.0167,0.0202,0.0197,0.0193,0.0179,0.0000,0.0000,0.0000,0.0000,0.0000,0.2779,0.0033,0.2374,0.0022
9000,0.0198,0.0248,0.0213,0.0210,0.0215,0.0230,0.0000,0.0000,0.0000,0.0000,0.0000,0.2743,0.0588,0.2376,0.0420


### 3

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.003$$
$$n_{epochs} = 10$$
$$BatchSize=50$$

In [ ]:
df6 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size)

[I 2025-08-17 18:49:28,153] A new study created in memory with name: no-name-d5061523-9cad-4d43-a177-edb90d613796
Best trial: 1. Best value: 0.114287:  20%|██        | 2/10 [00:13<00:43,  5.42s/it]  

[I 2025-08-17 18:49:41,075] Trial 0 finished with value: -0.04472256198718548 and parameters: {'lr': 0.07127227807002877, 'num_epochs': 5}. Best is trial 0 with value: -0.04472256198718548.
[I 2025-08-17 18:49:41,251] Trial 1 finished with value: 0.1142872512698241 and parameters: {'lr': 0.011825759151589394, 'num_epochs': 7}. Best is trial 1 with value: 0.1142872512698241.


Best trial: 1. Best value: 0.114287:  30%|███       | 3/10 [00:26<01:03,  9.06s/it]

[I 2025-08-17 18:49:54,627] Trial 2 finished with value: 0.045004589209512315 and parameters: {'lr': 0.014394419357596071, 'num_epochs': 10}. Best is trial 1 with value: 0.1142872512698241.


Best trial: 3. Best value: 0.135547:  40%|████      | 4/10 [00:27<00:34,  5.69s/it]

...

Best trial: 2. Best value: 0.0719043:  70%|███████   | 7/10 [02:21<01:05, 21.98s/it]

[I 2025-08-17 18:56:10,319] Trial 6 finished with value: -0.0051989051297017565 and parameters: {'lr': 0.006225043298701339, 'num_epochs': 8}. Best is trial 2 with value: 0.07190425393452309.


Best trial: 2. Best value: 0.0719043:  80%|████████  | 8/10 [02:23<00:35, 17.92s/it]

[I 2025-08-17 18:56:11,729] Trial 7 finished with value: 0.06390395626225005 and parameters: {'lr': 0.0026362426348245987, 'num_epochs': 7}. Best is trial 2 with value: 0.07190425393452309.


KeyboardInterrupt: 

In [ ]:
df6

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.0140,0.0206,0.0158,0.0163,0.0170,0.0190,0.0000,0.0000,0.0000,0.0000,0.0000,0.2189,0.0000,0.3154,0.0000
3000,0.0139,0.0206,0.0158,0.0163,0.0170,0.0190,0.0000,0.0000,0.0000,0.0000,0.0000,0.2189,0.0000,0.3154,0.0000
6000,0.0140,0.0156,0.0129,0.0135,0.0135,0.0136,0.0000,0.0000,0.0000,0.0000,0.0000,0.2079,0.0600,0.3195,0.0415
8000,0.0138,0.0194,0.0155,0.0168,0.0167,0.0163,0.0000,0.0000,0.0000,0.0000,0.0000,0.2189,0.0001,0.3154,0.0001
9000,0.0143,0.0164,0.0150,0.0165,0.0160,0.0146,0.0000,0.0000,0.0000,0.0000,0.0000,0.2155,0.0121,0.3156,0.0089


### 4

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.05$$
$$n_{epochs} = 10$$
$$BatchSize=150$$

In [ ]:
df7 = trainer_trial(num_runs, num_neighbors, num_rounds_list[:-3], train_dataset, batch_size+100, num_epochs=10, lr=0.05)

TypeError: trainer_trial() got an unexpected keyword argument 'num_epochs'

In [ ]:
df7